# 循环神经网络的简洁实现
本节将展示如何使用深度学习框架的高级API提供的函数更有效地实现相同的语言模型。我们从读取时光机器数据集开始。

## 读取序列数据
首先解决如何构建数据迭代器的问题，即随机生成一个小批量数据的特征和标签。由于文本序列可以是任意长的，例如整本《时光机器》（*The Time Machine*）。因此我们可以将文本序列划分为具有相同步数的子序列，从而在训练模型时，这样的子序列就可以分批输入。假设网络一次只处理具有$n$个时间步的子序列。下图画出了从原始文本序列获得子序列的所有不同的方式，其中$n=5$，并且每个时间步的词元对应于一个字符。


![分割文本时，不同的偏移量会导致不同的子序列](./timemachine-5gram.svg)

然而，如果我们只选择一个偏移量，那么用于训练网络的、所有可能的子序列的覆盖范围将是有限的。因此，我们可以从随机偏移量开始划分序列，以同时获得*覆盖性*（coverage）和*随机性*（randomness）。下面，我们将描述如何实现*随机采样*（random sampling）和*顺序分区*（sequential partitioning）策略。

### 随机采样
在随机采样中，每个样本都是在原始的长序列上任意捕获的子序列。在迭代过程中，来自两个相邻的、随机的、小批量中的子序列不一定在原始序列中相邻。对于语言模型，目标是基于到目前为止我们看到的词元来预测下一个词元，因此标签是移位了一个词元的原始序列。下面的代码每次可以从数据中随机生成一个小批量。在这里，参数`batch_size`指定了每个小批量中子序列样本的数目，参数`num_steps`是每个子序列中预定义的时间步数。

In [263]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l
import random
import math

In [264]:
def seq_data_iter_random(corpus, batch_size, num_steps):
    """使用随机抽样生成一个小批量子序列"""
    # 从随机偏移量开始对序列进行分区，随机范围包括num_steps-1
    corpus = corpus[random.randint(0, num_steps - 1):]
    # 减去1，是因为我们需要考虑标签
    num_subseqs = (len(corpus) - 1) // num_steps
    # 长度为num_steps的子序列的起始索引
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # 在随机抽样的迭代过程中，
    # 来自两个相邻的、随机的、小批量中的子序列不一定在原始序列上相邻
    random.shuffle(initial_indices)

    def data(pos):
        # 返回从pos位置开始的长度为num_steps的序列
        return corpus[pos: pos + num_steps]

    num_batches = num_subseqs // batch_size  # 
    for i in range(0, batch_size * num_batches, batch_size):
        # 在这里，initial_indices包含子序列的随机起始索引
        initial_indices_per_batch = initial_indices[i: i + batch_size]
        X = [data(j) for j in initial_indices_per_batch]
        # X = [corpus[pos: pos + num_steps] for pos in initial_indices_per_batch]
                
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield torch.tensor(X), torch.tensor(Y)

下面我们[**生成一个从$0$到$34$的序列**]。假设批量大小为$2$，时间步数为$5$，这意味着可以生成$\lfloor (35 - 1) / 5 \rfloor= 6$个“特征－标签”子序列对。如果设置小批量大小为$2$，我们只能得到$3$个小批量。

In [265]:
my_seq = list(range(35))
for X, Y in seq_data_iter_random(my_seq, batch_size=2, num_steps=5):
    print('X: ', X, '\nY:', Y)

X:  tensor([[15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24]]) 
Y: tensor([[16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25]])
X:  tensor([[ 0,  1,  2,  3,  4],
        [25, 26, 27, 28, 29]]) 
Y: tensor([[ 1,  2,  3,  4,  5],
        [26, 27, 28, 29, 30]])
X:  tensor([[ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14]]) 
Y: tensor([[ 6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15]])


### 顺序分区
在迭代过程中，除了对原始序列可以随机抽样外，我们还可以[**保证两个相邻的小批量中的子序列在原始序列上也是相邻的**]。这种策略在基于小批量的迭代过程中保留了拆分的子序列的顺序，因此称为顺序分区。

In [266]:
def seq_data_iter_sequential(corpus, batch_size, num_steps):  #@save
    """使用顺序分区生成一个小批量子序列"""
    # 从随机偏移量开始划分序列
    offset = random.randint(0, num_steps)
    num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
    Xs = torch.tensor(corpus[offset: offset + num_tokens])
    Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens])
    Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1)
    num_batches = Xs.shape[1] // num_steps
    for i in range(0, num_steps * num_batches, num_steps):
        X = Xs[:, i: i + num_steps]
        Y = Ys[:, i: i + num_steps]
        yield X, Y

基于相同的设置，通过顺序分区[**读取每个小批量的子序列的特征`X`和标签`Y`**]。通过将它们打印出来可以发现：迭代期间来自两个相邻的小批量中的子序列在原始序列中确实是相邻的。

In [267]:
my_seq = list(range(35))
for X, Y in seq_data_iter_sequential(my_seq, batch_size=2, num_steps=5):
    print('X: ', X, '\nY:', Y)

X:  tensor([[ 2,  3,  4,  5,  6],
        [18, 19, 20, 21, 22]]) 
Y: tensor([[ 3,  4,  5,  6,  7],
        [19, 20, 21, 22, 23]])
X:  tensor([[ 7,  8,  9, 10, 11],
        [23, 24, 25, 26, 27]]) 
Y: tensor([[ 8,  9, 10, 11, 12],
        [24, 25, 26, 27, 28]])
X:  tensor([[12, 13, 14, 15, 16],
        [28, 29, 30, 31, 32]]) 
Y: tensor([[13, 14, 15, 16, 17],
        [29, 30, 31, 32, 33]])


现在，我们[**将上面的两个采样函数包装到一个类中**]，以便稍后可以将其用作数据迭代器。

In [268]:
class SeqDataLoader:
    """加载序列数据的迭代器"""
    def __init__(self, batch_size, num_steps, use_random_iter, max_tokens):
        if use_random_iter:
            self.data_iter_fn = seq_data_iter_random
        else:
            self.data_iter_fn = seq_data_iter_sequential
        self.corpus, self.vocab = d2l.load_corpus_time_machine(max_tokens)
        self.batch_size, self.num_steps = batch_size, num_steps
    def __iter__(self):
        return self.data_iter_fn(self.corpus, self.batch_size, self.num_steps)

[**最后，我们定义了一个函数`load_data_time_machine`，它同时返回数据迭代器和词表**]

In [269]:
def load_data_time_machine(batch_size, num_steps,
                           use_random_iter=False, max_tokens=10000):
    """返回时光机器数据集的迭代器和词表"""
    data_iter = SeqDataLoader(
        batch_size, num_steps, use_random_iter, max_tokens)
    return data_iter, data_iter.vocab

### 读取数据

In [270]:
batch_size, num_steps = 32, 35
train_iter, vocab = load_data_time_machine(batch_size, 
                            num_steps,
                            use_random_iter=False, 
                            max_tokens=10000)

In [271]:
len(vocab)

28

In [272]:
next(iter(train_iter))[0].shape

torch.Size([32, 35])

In [273]:
next(iter(train_iter))[1].shape

torch.Size([32, 35])

## 定义模型
高级API提供了循环神经网络的实现，我们构造一个具有256个隐藏单元的单隐藏层的循环神经网络层`rnn_layer`。

In [274]:
num_hiddens = 256
rnn_layer = nn.RNN(len(vocab), num_hiddens)

我们(**使用张量来初始化隐状态**)，它的形状是（隐藏层数，批量大小，隐藏单元数）。

In [275]:
state = torch.zeros((1, batch_size, num_hiddens))
state.shape

torch.Size([1, 32, 256])

通过一个隐状态和一个输入，我们就可以用更新后的隐状态计算输出。需要强调的是，`rnn_layer`的输出（`Y`）不涉及输出层的计算：它是指每个时间步的隐状态，这些隐状态可以用作后续输出层的输入。

In [276]:
X = torch.rand(size=(num_steps, batch_size, len(vocab)))
Y, state_new = rnn_layer(X, state)

Y是最后一层每一个时间步每个样本的隐状态集合

In [277]:
Y.shape

torch.Size([35, 32, 256])

state_new是最后一个时间步每一层每个样本的隐状态集合

In [278]:
state_new.shape

torch.Size([1, 32, 256])

[**我们为一个完整的循环神经网络模型定义了一个`RNNModel`类**]。注意，`rnn_layer`只包含隐藏的循环层，我们还需要创建一个单独的输出层。


In [279]:
class RNNModel(nn.Module):
    """循环神经网络模型"""
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        # 如果RNN是双向的（之后将介绍），num_directions应该是2，否则应该是1
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)  # Y的形状为：时间步数，批量大小，隐藏单元数
        # 全连接层首先将Y的形状改为（时间步数*批量大小， 隐藏单元数）
        output = self.linear(Y.reshape(-1, Y.shape[-1]))
        return output, state
    
    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                        self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))

In [280]:
net = RNNModel(rnn_layer, len(vocab))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=1)
net.to(device)

RNNModel(
  (rnn): RNN(28, 256)
  (linear): Linear(in_features=256, out_features=28, bias=True)
)

## 训练与预测

In [281]:
def trian_rnn(net, train_iter, num_epochs, loss, 
                                  optimizer, device):
    net.to(device)
    for epoch in range(num_epochs):
        state = None
        loss_total = 0
        num_batches = 0
        for X , Y in train_iter:
            # 第一次迭代初始化
            if state is None:
                state = net.begin_state(device=device, batch_size=X.shape[0])
            else:
                state.detach_()  # 除第一次迭代，state可以连用，只需去掉梯度信息
                
            y = Y.T.reshape(-1)
            X, y = X.to(device), y.to(device)
            y_hat, state = net(X, state)

            l = loss(y_hat, y.long())
            optimizer.zero_grad()
            l.backward()
            
            # 梯度裁剪
            
            params = [p for p in net.parameters() if p.requires_grad]
            norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
            if norm > 1:
                for param in params:
                    param.grad[:] *= 1 / norm
            
            optimizer.step()

            loss_total += l * y.numel()
            num_batches += y.numel()
        if epoch % 100 == 0:
            print(f'perplexity in epoch {epoch}: {math.exp(loss_total / num_batches)}')  

In [282]:
trian_rnn(net, train_iter, 1000, loss, optimizer, device)

perplexity in epoch 0: 21.809054336601108
perplexity in epoch 100: 3.53940942681019
perplexity in epoch 200: 1.595776524082216
perplexity in epoch 300: 1.3879890933519314
perplexity in epoch 400: 1.353919754472193
perplexity in epoch 500: 1.2952187097994179
perplexity in epoch 600: 1.2969301902518724
perplexity in epoch 700: 1.2677194429967487
perplexity in epoch 800: 1.3034602467949956
perplexity in epoch 900: 1.3408058268501963


In [283]:
def predict(prefix, num_preds, net, vocab, device):
    """在prefix后面生成新字符"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测num_preds步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    print(''.join([vocab.idx_to_token[i] for i in outputs])) 

In [287]:
predict('time traveller ', 50, net, vocab, device)

time traveller come than upast really this incumertal is the fiou
